## PT-1 demonstration notebook

This notebook demonstrates the basic functionalities of the PT-1, including how to connect, perform experiments, and run machine learning algorithms. Before using this notebook make sure that the PT-1 hardware is ready by following the device's manual. The current status of the PT Series hardware is indicated on the status page.

**This notebook assumes you are running the SDK on the on-premise computer supplied with the PT-1.**


In [ ]:
import os

import numpy as np
import torch

if os.getcwd().endswith("notebooks"):
    os.chdir("..")

from ptseries.models import PTLayer
from ptseries.tbi import create_tbi

First, a connection to the device must be opened. We can do this by calling the `create_tbi` function with the `tbi_type` set to PT-1. Note that the object has some differences to the simulated backends you may have used previously. Specifically, you cannot specify parameters such as beam splitter loss because this is now a property of the device. For a full comparison, please consult the SDK documentation. Make sure to replace the value of `x.x.x.x` with the actual address.


In [ ]:
# You can either use the a previously set environment variable ORCA_ACCESS_URL (see the documentation)
# or pass the url directly to the tbi_params
tbi_params = {
    "tbi_type": "PT-1",
    "url": "http://x.x.x.x",
}

time_bin_interferometer = create_tbi(**tbi_params)

### Sampling

Once the above methods tell you the hardware is ready to use, you can run your first experiment on this boson sampler! This can be done by calling the sample method. Note that the number of beam splitter angles, specified by 'theta_list' will always be the number of modes minus one.

The next two cells draw the circuit representation of the PT-1 for our target state and then produce 200 samples from it.


In [ ]:
input_state = (1, 0, 0)  # 1 photon in 3 modes
theta_list = [np.pi / 4] * (len(input_state) - 1)  # 50/50 beam splitters
n_samples = 200

time_bin_interferometer.draw(input_state=input_state)

In [ ]:
samples = time_bin_interferometer.sample(
    input_state=input_state,
    theta_list=theta_list,
    n_samples=n_samples,
)
print(samples)

Congratulations, you have just created and measured a quantum state where the single photon has roughly 50% probability of being measured in the first mode, and 25% probability of being measured in the two other modes. Deviations from these probabilities are caused by experimental imperfections such as loss in the loop.


### Training a PT-1

The angles that define the reflectivity of the beam splitters are trainable using PyTorch. To do this we must define a model, analogous to a neural network, called PTlayer. This model uses a custom extension of PyTorch's Autograd to interact with the PT-1.
In this example we use this feature to train the PT-1 to route a photon from the first mode to the last mode. First we define the model, with a single photon input into the first mode, and view the output of the model.


In [ ]:
model = PTLayer(
    (1, 0, 0, 0),  # The input state
    observable="avg-photons",
    in_features=0,
    n_samples=100,
    tbi_params=tbi_params,
)

model.eval()
outputs_initial = model()
print(outputs_initial)

Note that the beam splitter values in PTLayer are initialised randomly, so the input photon initially gets randomly distributed among all four modes.

Next we define how we train the model. Because the PT-1 uses PyTorch, you can use optimizers and loss functions just as you would for a classical model. Here we also define our objective, which is routing the photon into the last mode.


In [ ]:
iterations = 100
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
objective = torch.tensor([[0.0, 0.0, 0.0, 1.0]], dtype=torch.float32)

loss = torch.nn.MSELoss()

We then train the model for a given number of epochs, resetting the gradients at each step. The output from the model is now a photon routed to the last mode.


In [ ]:
model.train()

for i in range(iterations):
    optimizer.zero_grad()
    loss_value = loss(model(), objective)
    loss_value.backward()
    optimizer.step()
    if i % 10 == 0:
        print("Loss at iteration {}: ".format(i), loss_value.item())

model.eval()
outputs_final = model()
print("Final output state: ", outputs_final)

### Saving Data

You can automatically save data from the PT-1 by specifying save_dir when running the sampling command. This can be useful if you would like to export the sample for analysis.


In [ ]:
samples = time_bin_interferometer.sample(
    input_state=(1, 0, 0, 0), theta_list=[np.pi / 4] * 3, n_samples=100, save_dir="data"
)

# Variational Classifier

For an example showing how to use the PT-1 for classification, we suggest that you open and run the `quantum_variational_classification` notebook. To run this model on the PT-1, simply modify the line where you instantiate the model to `model = Model(tbi_params={"tbi_type": "PT-1", "url": "http://x.x.x.x"})`
